
<br>
This is a demo file to show how to use the townhall package.<br>


In [1]:
from townhall.agents.planner import Planner
from townhall.agents.python_engineer import PythonEngineer
from townhall.agents.python_qa import PythonQA
# from townhall.agents.product_manager import ProductManager
# from townhall.agents.critic import Critic
from townhall.agents.user_agent import UserAgent
from townhall.agents.scientist import Scientist
from townhall.services.chat_service import ChatService
from settings import CONFIG_LIST

In [2]:
executor = UserAgent(
    name="Executor",
    system_message="Executor. Execute the code written by any agent and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": "3", "work_dir": "agent_workplace"},
)

In [3]:
CHAT_SERVICE = ChatService(
    config_list=CONFIG_LIST,
    assistants=[
       PythonEngineer(system_message='''Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
'''),
       Scientist(system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code."""),
       Planner(system_message='''Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
'''), 
       executor,
       PythonQA(system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL."),
    ],
    user_proxy=UserAgent(name="Admin", code_execution_config=False, system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.")
)

In [4]:
HEADER_MESSAGE_TO_USER = """
Note: We can run out of context length. Autogen are not able to handle this yet,
we are looking at ways of integrating llama_index to solve this problem.
Enter a message to send to the Research Team:
"""

In [5]:
text_input = """
find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.
"""
try:
    CHAT_SERVICE.initiate_chat(text_input)
except:
    message = CHAT_SERVICE.messages[-1:]
    CHAT_SERVICE.initiate_chat(f"We have encoured an error, try and reduce the length of your message. \n {message}", clear_history=True)

Admin (to chat_manager):


find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.


--------------------------------------------------------------------------------

find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.


--------------------------------------------------------------------------------
Planner (to chat_manager):

Plan:
1. Engineer: Write a script to scrape the arXiv website and retrieve papers on LLM applications from the last week.
2. Engineer: Extract relevant information from the scraped papers, such as the title, authors, and abstract.
3. Scientist: Review the extracted information and categorize the papers into different domains based on their content.
4. Scientist: Create a markdown table with the domains as columns and populate it with the relevant papers.
5. Engineer: Generate a markdown file with the table and save it.

Please note that the engineer will be 